In [1]:
import json
import os
import random
import socket
import sys
import pandas as pd
import numpy as np

sys.path.append("/Users/hashiguchiyutaka/Desktop/3S/prog/submarine-py")

from lib.make_coordinates import make_all_coordinates, valid_coordinates, make_not_near_coordinates, all_nears, choose_nearest, distance, make_near_x_or_y, center_coordinates
from lib.player_base import Player, PlayerShip

import ipytest
ipytest.autoconfig()

In [2]:
class EnemyShip(PlayerShip):

    # 種類と場所を与えられる．HPは自動で決まる．
    def __init__(self, ship_type, position=None):
        if ship_type not in PlayerShip.MAX_HPS:
            raise ValueError('invalid type supecified')

        self.type = ship_type
        self.position = position
        self.hp = PlayerShip.MAX_HPS[ship_type]

    # ダメージを受けてHPが減る．
    def damaged(self, d):
        self.hp -= d

    # 座標を変更する．
    # 敵の船については、場所が確定しない限り使えない
    def moved(self, to):
        self.position = to

    # 座標が移動できる範囲(縦横)にあるか確認する．
    def can_reach(self, to):
        if self.position is None:
            raise AttributeError('You cannot specify the position')
        return self.position[0] == to[0] or self.position[1] == to[1]

    # 座標が攻撃できる範囲(自分の座標及び周囲1マス)にあるか確認する．
    def can_attack(self, to):
        if self.position is None:
            raise AttributeError('You cannot specify the position')
        return abs(to[0] - self.position[0]) <= 1\
            and abs(to[1] - self.position[1]) <= 1

In [ ]:
class Enemy(Player):
    def __init__(self):
        self.df = pd.DataFrame(make_all_coordinates()) # 盤面の全パターンを初期値に
        # self.hp = {"w": 3, "c": 2, "s": 1} 相手のhpは今のところ不要では？
        self.ships = ["w", "c", "s"]

    def remove_same_position(self):
        """
        self.ships内の船の位置が重複している行を削除する
        """
        # 船が1つしかなければ重複はない
        if len(self.ships) == 1:
            return None
        elif len(self.ships) == 2:
            self.df = self.df[self.df.apply(lambda x: x[self.ships[0]] != x[self.ships[1]], axis=1)]
        else:
            self.df = self.df[self.df.apply(lambda x: x["w"] != x["c"] and x["w"] != x["s"] and x["c"] != x["s"], axis=1)]

        # 0になったらself.shipsからその船を消す
        # 直接消してしまうと、remove_same_position(self.df)などの実装で不具合があるので、probを求める際に消えている船の座標はカウントしないことにする
        # と思ったけど、こうすると、remove_same_positionで、死んだ船と被っている行動も不可能と見做されてしまう
        # 他の船の位置の組に重複があるが、確率を考える上では問題ない

    def move(self, ship_type, direction) -> None:
        """
        敵のmoveを処理し、self.dfを更新
        direction = [1,0]などの移動分を表すベクトル
        self.df.loc[:, ship_type]を使うことでSettingWithCopyWarningをさけ、確実に元のdfを変更できるようにする
        """
        delta_x = direction[0]
        delta_y = direction[1]
        self.df.loc[:, ship_type] = self.df[ship_type].map(lambda x: (x[0] + delta_x , x[1] + delta_y))
        
        
        # 有効な座標以外の盤面を削除
        self.df = self.df[self.df[ship_type].map(lambda x: x in valid_coordinates())]

        # 座標がダブっている船を含む盤面を削除(移動によりダブってしまうことがある)
        self.remove_same_position()
    
    def attack(self, position) -> None:
        """
        敵のattackを処理し、self.dfを更新
        position = [1,0]などの相手の攻撃した座標
        positionの周囲1マス(自身を含む)にいずれかの敵がいることが確定する
        """
        near_positions = all_nears(position, me=True)

        # self.dfの中から、near_positionsに含まれる座標を持つ行のみを残す
        # ただし、この時点で敵のw,c,sが全て存在するとは限らないことに注意

        # self.dfの行が、near_positionsに含まれる座標を持つかどうかを判定する関数
        def is_valid(x): # xはself.dfの行
            for ship in self.ships:
                if x[ship] in near_positions:
                    return True
            # どの船もnear_positionsに含まれる座標を持っていない場合
            return False
            
        self.df = self.df[self.df.apply(is_valid, axis=1)]

    def hit(self, ship_type, position) -> None:
        """
        自分の攻撃がhitして、かつその船がまだ生きている時の処理
        """
        coordinate = tuple(position) # 忘れずに
        self.df = self.df[self.df[ship_type] == coordinate]
        # print(self.prob()) hit関数が動いているなら、ここを表示すると常に確率1の座標が存在するはず

    def miss(self, position) -> None:
        """
        自分の攻撃がmissした時の処理
        """
        # 結局、near/not_nearで処理するので何もしない
        pass

    def near(self, ship_type, position) -> None:
        """
        自分の攻撃の結果、ship_typeがnearにいた時の処理
        positionの周囲1マス(自身を含まない)にその船がいることが確定する
        """
        coordinate = tuple(position)
        # near_positionsはpositionの周囲1マス(自身を含まない)の座標
        near_positions = all_nears(coordinate, me=False)
        self.df = self.df[self.df[ship_type].map(lambda x: x in near_positions)]

    def not_near(self, ship_type, position) -> None:
        """ 
        自分の攻撃の結果、ship_typeがnot_nearだった時の処理
        positionの周囲1マス(自身を含む)以外のマスにその船がいることが確定する
        """
        coordinate = tuple(position)
        # near_positionsはpositionの周囲1マス(自身を含む)の座標
        near_positions = all_nears(coordinate, me=True)
        self.df = self.df[self.df[ship_type].map(lambda x: x not in near_positions)]

    def safe_position(self) -> set:
        """
        相手の船が絶対にいない座標のsetを返す
        """
        not_safe = set() # {}とするとdictになるので注意
        for coordinate in self.prob().keys():
            # coordinateの周囲1マス(自身を含む)の座標をnot_safeに追加
            not_safe = not_safe | all_nears(coordinate, me=True)
        # valid_coordinates()からnot_safeを引いたものが安全な座標
        return valid_coordinates() - not_safe # setの差集合
    
    def prob(self):
        return self.df.stack().value_counts()/len(self.df)
    
    def where_to_attack(self):
        """
        self.dfの中から、確率高い座標(tuple)を順に返す
        [(4, 2),(4, 0),...]
        """
        return [coordinate for coordinate in self.prob().keys()]

In [32]:
enemy = Enemy()
enemy.attack([0,0])
enemy.prob()

(1, 0)    0.284536
(0, 1)    0.284536
(0, 0)    0.284536
(1, 1)    0.284536
(4, 0)    0.088660
(1, 2)    0.088660
(1, 4)    0.088660
(2, 0)    0.088660
(0, 3)    0.088660
(4, 4)    0.088660
(4, 1)    0.088660
(3, 2)    0.088660
(2, 1)    0.088660
(0, 4)    0.088660
(3, 0)    0.088660
(2, 4)    0.088660
(3, 3)    0.088660
(3, 4)    0.088660
(4, 2)    0.088660
(1, 3)    0.088660
(2, 2)    0.088660
(0, 2)    0.088660
(3, 1)    0.088660
(4, 3)    0.088660
(2, 3)    0.088660
Name: count, dtype: float64

In [34]:
enemy.attack([4,0])
enemy.prob()

(4, 0)    0.26875
(1, 1)    0.26875
(4, 1)    0.26875
(3, 1)    0.26875
(3, 0)    0.26875
(0, 0)    0.26875
(0, 1)    0.26875
(1, 0)    0.26875
(0, 4)    0.05000
(1, 4)    0.05000
(2, 0)    0.05000
(0, 3)    0.05000
(4, 4)    0.05000
(3, 2)    0.05000
(2, 1)    0.05000
(1, 2)    0.05000
(4, 3)    0.05000
(2, 4)    0.05000
(3, 3)    0.05000
(3, 4)    0.05000
(4, 2)    0.05000
(1, 3)    0.05000
(2, 2)    0.05000
(0, 2)    0.05000
(2, 3)    0.05000
Name: count, dtype: float64

In [35]:
enemy.attack([1,4])

In [36]:
enemy.prob()

(4, 0)    0.250000
(1, 0)    0.250000
(0, 1)    0.250000
(0, 0)    0.250000
(1, 1)    0.250000
(3, 1)    0.250000
(3, 0)    0.250000
(4, 1)    0.250000
(1, 3)    0.166667
(2, 4)    0.166667
(0, 4)    0.166667
(0, 3)    0.166667
(1, 4)    0.166667
(2, 3)    0.166667
Name: count, dtype: float64

In [37]:
enemy.attack([2,4])
enemy.prob()

(4, 0)    0.25
(1, 0)    0.25
(1, 3)    0.25
(2, 4)    0.25
(1, 4)    0.25
(2, 3)    0.25
(0, 1)    0.25
(0, 0)    0.25
(1, 1)    0.25
(3, 1)    0.25
(3, 0)    0.25
(4, 1)    0.25
Name: count, dtype: float64

In [38]:
enemy.attack([4,2])
enemy.prob()

(3, 1)    0.50
(4, 1)    0.50
(1, 0)    0.25
(1, 3)    0.25
(2, 4)    0.25
(1, 4)    0.25
(2, 3)    0.25
(0, 1)    0.25
(0, 0)    0.25
(1, 1)    0.25
Name: count, dtype: float64

In [39]:
enemy.attack([1,4])
enemy.prob()

(3, 1)    0.50
(4, 1)    0.50
(1, 0)    0.25
(1, 3)    0.25
(2, 4)    0.25
(1, 4)    0.25
(2, 3)    0.25
(0, 1)    0.25
(0, 0)    0.25
(1, 1)    0.25
Name: count, dtype: float64

In [21]:
enemy.move("w", [4, 0])

,w,c,s
6101,"(4, 1)","(3, 4)","(1, 3)"
6107,"(4, 1)","(3, 4)","(0, 4)"
6114,"(4, 1)","(3, 4)","(0, 3)"
6116,"(4, 1)","(3, 4)","(1, 4)"
6124,"(4, 1)","(4, 3)","(1, 3)"
...,...,...,...
12039,"(4, 3)","(0, 0)","(3, 3)"
12047,"(4, 3)","(0, 0)","(4, 4)"
12053,"(4, 3)","(1, 1)","(3, 4)"
12062,"(4, 3)","(1, 1)","(3, 3)"


In [22]:
enemy.prob()

(4, 3)    0.428571
(4, 4)    0.428571
(4, 1)    0.285714
(3, 4)    0.285714
(3, 3)    0.285714
(4, 0)    0.285714
(1, 3)    0.142857
(0, 4)    0.142857
(0, 3)    0.142857
(1, 4)    0.142857
(1, 0)    0.107143
(0, 1)    0.107143
(0, 0)    0.107143
(1, 1)    0.107143
Name: count, dtype: float64

In [11]:
# OK
enemy = Enemy()
enemy.move("w", [4, 0])
enemy.prob()

(4, 2)    0.269565
(4, 0)    0.269565
(4, 3)    0.269565
(4, 4)    0.269565
(4, 1)    0.269565
(2, 3)    0.086957
(1, 4)    0.086957
(2, 0)    0.086957
(1, 1)    0.086957
(3, 2)    0.086957
(2, 1)    0.086957
(1, 2)    0.086957
(2, 4)    0.086957
(3, 3)    0.086957
(3, 0)    0.086957
(1, 3)    0.086957
(1, 0)    0.086957
(2, 2)    0.086957
(3, 1)    0.086957
(3, 4)    0.086957
(0, 4)    0.069565
(0, 1)    0.069565
(0, 0)    0.069565
(0, 3)    0.069565
(0, 2)    0.069565
Name: count, dtype: float64

## 座標の取り出し

In [15]:
enemy.prob().keys().tolist()

[(4, 2),
 (4, 0),
 (4, 3),
 (4, 4),
 (4, 1),
 (2, 3),
 (1, 4),
 (2, 0),
 (1, 1),
 (3, 2),
 (2, 1),
 (1, 2),
 (2, 4),
 (3, 3),
 (3, 0),
 (1, 3),
 (1, 0),
 (2, 2),
 (3, 1),
 (3, 4),
 (0, 4),
 (0, 1),
 (0, 0),
 (0, 3),
 (0, 2)]

In [16]:
[list(coordinate) for coordinate in enemy.prob().keys().tolist()]

[[4, 2],
 [4, 0],
 [4, 3],
 [4, 4],
 [4, 1],
 [2, 3],
 [1, 4],
 [2, 0],
 [1, 1],
 [3, 2],
 [2, 1],
 [1, 2],
 [2, 4],
 [3, 3],
 [3, 0],
 [1, 3],
 [1, 0],
 [2, 2],
 [3, 1],
 [3, 4],
 [0, 4],
 [0, 1],
 [0, 0],
 [0, 3],
 [0, 2]]

## 初期の実験

In [ ]:
enemy = Enemy()
enemy.prob()
enemy.move("w", "r", 3)
#print(enemy.df["c"].value_counts())
enemy.move("w", "r", 1)
#print(enemy.df["c"].value_counts())
enemy.move("w", "d", 4)
print(enemy.df["c"].value_counts())
enemy.move("c", "r", 3)
enemy.df["c"].value_counts()
enemy.move("c", "d", 3)
enemy.df["c"].value_counts()
enemy.prob()
# dictカラムに、{'w': [0, 0], 'c': [0, 1], 's': [0, 2]}のような形式で座標を格納
# dataframe全体にapplyするときは、axis=1を指定する
df["dict"] = df.apply(lambda x: {"w": x["w"], "c": x["c"], "s": x["s"]}, axis=1)
### テスト1
enemy = Enemy(df_start)
enemy.move("s", "l", 2)
enemy.prob()
enemy.move("c", "r", 2)
enemy.prob()
enemy.move("c", "l", 1)
enemy.prob()
enemy.move("w", "r", 2)
enemy.prob()
enemy.move("c", "u", 4)
enemy.prob()
enemy.move("c", "d", 4)
enemy.prob()
enemy.move("w", "l", 1)
enemy.prob()
enemy.move("w", "d", 2)
enemy.prob()
enemy.move("w", "u", 1)
enemy.prob()
enemy.move("w", "d", 1)
enemy.prob()
enemy.move("w", "u", 2)
enemy.prob()
enemy.move("w", "d", 1)
enemy.df["w"].value_counts()
enemy.move("w", "r", 1)
enemy.df["w"].value_counts()
enemy.move("w", "l", 2)
enemy.df["w"].value_counts()
enemy.move("w", "r", 1)
enemy.df["w"].value_counts()
enemy.move("w", "l", 3)
enemy.df["w"].value_counts()
enemy.move("w", "u", 1)
enemy.df["w"].value_counts()
enemy.move("w", "r", 4)
enemy.df["w"].value_counts()
### テスト2
- 船を消す実装をした
enemy = Enemy(df_start)
enemy.move("s", "l", 2)
enemy.prob()
enemy.damaged("s")
enemy.ships
enemy.prob()
enemy.damaged("c")
enemy.prob()
enemy.move("c", "r", 2)
enemy.prob()
enemy.move("c", "l", 1)
enemy.prob()
enemy.damaged("c")
enemy.prob()
enemy.hp
enemy.move("w", "r", 2)
enemy.prob()
enemy.move("w", "l", 3)
enemy.prob()
enemy.move("w", "r", 2)
enemy.prob()